# TF-IDF: different results from different definitions
by MAS, 2018

Last year, I wrote a [Python script](https://github.com/mas16/keywords) to scrape text from the [NIH database](https://www.ncbi.nlm.nih.gov/pubmed/) of published research and classify different researchers by the words that appeared most frequently across their publications. The script calculates a statistic called the **term frequency** (tf) which is the number of times a given word appears in a given text. Since I wasn't too careful about style back then, my code is a bit clunky so I decided to see if I could streamline things with scikit-learn (```sklearn```).

I hadn’t used ```sklearn``` for text analysis before so I started by looking for a good tutorial. After some googling, I found Chris Perone’s awesome [post](http://blog.christianperone.com/2011/09/machine-learning-text-feature-extraction-tf-idf-part-i/) about ```sklearn```’s text feature extraction methods. He covers the tf which is what I was originally interested in as well as an additional statistic known as the **inverse document frequency** (idf). The product of the tf and idf (tf-idf) is a good reporter for how important a word is across *many* texts so I thought it’d be useful to learn about it. 

I worked through the tutorial (more than once to make sure I was doing it correctly) and was surprised that I couldn’t reproduce his results. This may have been because the post is 8 years old and ```sklearn``` has changed its implementation of things. **Regardless, after some troubleshooting, I found out that ```sklearn``` defines the idf a little bit differently than what is described elsewhere.** 

If you are not familiar with the tf and idf, I’d highly recommend you check out Chris’ post first. Wikipedia also has a pretty good [description](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) with some examples.

Let’s start with Chris’ example and work through what ```sklearn``` does for us. 

First, define the training and test sets that Chris provides:

In [8]:
# TRAIN SET

# d1: the sky is blue
# d2: the sun is bright

train_set = ["The sky is blue.", "The sun is bright."]

# TEST SET

# d3: The sun in the sky is bright
# d4: We can see the shining sun, the bright sun

test_set = ["The sun in the sky is bright.",
            "We can see the shining sun, the bright sun."]

The purpose of the training set is to define a list of key words (vocabulary) that we will look for in the test set. Once we have the words, we can count them and determine the tf for each. Words like “the” and “is” will be ignored since they are likely to appear all over the place and usually hold little information content. We can use ```sklearn``` to construct the vocabulary:

In [9]:
# import CountVectorizer from sklearn 

from sklearn.feature_extraction.text import CountVectorizer

# make instance of CountVectorizer called 'vectorizer'
# add stop_words='english' argument to ignore trivial words

vectorizer = CountVectorizer(stop_words='english')

# create the vocabulary
# the vocabulary is a python dictionary
# where the keys are the terms and the values are the indices
# corresponding to the terms' element after mapping to a vsm

vectorizer.fit_transform(train_set)
print(vectorizer.vocabulary_)

{'sky': 2, 'blue': 0, 'sun': 3, 'bright': 1}


The ```stop_words``` argument passed to ```CountVectorizer``` eliminates trivial words like "the" and "is." To keep those words in the analysis, just don't pass ```stop_words```.

As you can see from the code block above, the vocabulary is returned as a Python dictionary.

The keys are the non-trivial words from the training set that ```sklearn``` identified. The values are the indices for those words in the vector space that the text has been mapped to. This might look a little funny because the order is scrambled. If you want to view the vocabulary in the order that the words appear, you can do this:

In [10]:
# to get the words in order do: vectorizer.get_feature_names()
# order here refers to the order in which the terms appeared in the text

print(vectorizer.get_feature_names())

['blue', 'bright', 'sky', 'sun']


Next, let’s create the vector space for the test set and get the tfs:

In [11]:
# make a sparse matrix using transform method
# the matrix will contain the tf in the element position corresponding
# to the values listed in the vocabulary dictionary

smatrix = vectorizer.transform(test_set)

# use .todense() to print in matrix format not coordinate format

print(smatrix.todense())

[[0 1 1 1]
 [0 1 0 2]]


The two rows are for the two documents in the test set, ```d1``` and ```d2``` respectively. The element with index 0 is the number of times the word ‘blue’ appears, the element with index 1 is the number of times the word ‘bright’ appears, the element with index 2 is the number of times the word ‘sky’ appears, and the element with index 3 is the number of times the word ‘sun’ appears. **In other words, these are the tfs of the words from our vocabulary.** 

At first blush, this might look different from the result Chris posted but the only real difference is the indexing of the words. All of the tf values are the same.

Now to move on to the complicated part: calculating the idf. So the idf equation that is listed on Chris’ post is the textbook definition:

$ \Large { idf(t,D) = log \frac{D}{1+\{ d \in D  :  t \in d\}} }$

FYI Chris uses a log base e (ln) whereas wikipedia suggests using log base 10. 

What this equation is saying is that the idf value is determined by two parameters: the number of times a given word appears (t) and the total number of documents (D). The idf is then calculated as the log of the total number of documents (D) divided by 1 plus the number of documents (d) within D that contain the word (t).

Based on the comments on Chris' post, there seems to be a bit of confusion about the denominator term. Chris states that it is the number of documents that contain the word. A few people commented that the denominator was the number of times the word appeared across all documents. From my research, the former definition is correct.

Given the above information, we would expect to see the following results for the idf calculation:

  > **Blue**: $ln (2/(1+0)) = 0.693$

  > **Bright**: $ln (2/(1+2)) = -0.405$

  > **Sky**: $ln (2/(1+1)) = 0$

  > **Sun**: $ln (2/(1+2)) = -0.405$
  
 Which is what Chris reports. Now multiplying the idf values by the tf values to get the td-idf values should yield: 

$$
\large{
\begin{bmatrix} 
0 & -0.405 & 0 & -0.405 \\
0 & -0.405 & 0 & -0.810 
\end{bmatrix}
}
$$

Again, this is what Chris shows just with different element indices. Ok, so this is the answer we are looking for. Now let’s see what we get when we do the tf-idf calculation in ```sklearn```:

In [12]:
# Inverse document frequency (idf) feature extraction using sklearn
# sklearn's TfidfTransformer is used to calculate the idf for each
# term across all texts in test set. The default is to normalize all
# idf values by the L2 norm. This normalization has been disabled by
# passing the argument norm=None so that it is more intuitive to see
# what is going on and can be compared to Chris' post.

# import TfidfTransformer from sklearn

from sklearn.feature_extraction.text import TfidfTransformer

# make instance of TfidfTransformer called "tfidf"

tfidf = TfidfTransformer(norm=None)

# return un-normalized tf-idf matrix using sparse matrix of tf values that
# was generated above

tfidf_matrix = tfidf.fit_transform(smatrix)

print(tfidf_matrix.todense())

[[0.         1.         1.40546511 1.        ]
 [0.         1.         0.         2.        ]]


So this is quite different from what we expected. When stuff like this happens, I immediately go to the documentation page or stackoverflow (after first making sure that I didn’t make a mistake). **After some digging, I found out that the default equation for calculating the idf in sklearn is not the same as what is shown above!** The equation that sklearn uses is described [here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html):

> If ```smooth_idf=True``` (the default), the constant “1” is added to the numerator and denominator of the idf as if an extra document was seen containing every term in the collection exactly once, which prevents zero divisions: idf(d, t) = log [ (1 + n) / (1 + df(d, t)) ] + 1.

Now if we do this calculation using the formula above, we get the following:

   >**Blue**: $ln ((1+2)/(1+0)) + 1 = 2.099$

   >**Bright**: $ln ((1+2)/(1+2)) + 1= 1.0$

   >**Sky**: $ln ((1+2)/(1+1)) + 1= 1.405$

   >**Sun**: $ln ((1+2)/(1+2)) + 1 = 1.0$
   
The expected tf-idf would then be:

$$
\large{
\begin{bmatrix} 
0 & 1 & 1.405 & 1 \\
0 & 1 & 0 & 2 
\end{bmatrix}
}
$$

Which is exactly what we saw!

### So the moral of the story is…make sure you know which definition you are using!